# Project: automatic error propagation
### Michiel Stock, Bram De Jaegher, Daan Van Hauwermeiren
### 18 December 2019
# What is error propagation?

During my first year of college, I really liked the physics lectures because everything was exact. In my first year of physics, I really disliked the physics labs, because everything was messy. First-year physics labs are generally disliked partially because of the uninspiring topics (measuring resistance in a wire! determining a heat transfer coefficient! measuring chirality of sugar!) and partly because this was the only lab in which 'exact' measurements had to be performed. These labs introduced the complex and unexplained rules for measurement error propagation. Without any prior statistics or probability courses, it was lost on me where these strange rules originated from, as sharp contrast to nearly everything else in physics.

A year later, after plowing through a basic probability course, error propagation is less mysterious. Almost all these rules to account for the error can be derived from a simple principle:

> Given two **independent** random variables $$X$$ and $$Y$$, the variance of a linear combination $$\text{Var}[aX+bY]=a^2\text{Var}[X] + b^2\text{Var}[Y]$$.

Measurement errors are usually given by a standard deviation, the square root of the variance. Given this principle, error propagation is merely bookkeeping of the standard error on the measurement for various computations. This [table on error propagation](https://en.wikipedia.org/wiki/Propagation_of_uncertainty#Example_formulae) might be useful.

For nonlinear functions, we can compute an approximate uncertainty propagation using a first-order Taylor approximation. We have, for any function $f(x)$:

$$
f(x\pm \sigma) \approx f(x) \pm |f'(x)|\sigma\,.
$$

For example, for squaring a function, we have

$$
(x\pm\sigma)^2 = x^2 \pm 2|x|\sigma\,.
$$

Note that this is consistent with the above rules for multiplication. Let us implement the general formula for raising a measurement to the power $p$.

We can implement this for all the standard mathematical functions one by one. However, Julia provides use with two efficient tools to do this in one swoop: automatic differentiation and metaprogramming. We just loop a list of functions of interest and automatically generate the correct approximate rule.


Instead of processing the measurements and the standard error separately, suppose we could just make a new type of number which contains both the observed value and its uncertainty. And suppose we could just compute something with these numbers, plugging them into our formulas where the error is automatically accounted for using the standard error propagation rules. In Julia, it is dead simple to construct such new numbers and just overload existing functions such that they compute in the correct way.

# Goal of this project

We will implement a binary operator `±`, which can be used to add the standard error to a measurement.
The result is a new type `Measurement` containing both the value and the standard error. Standard functions will be overloaded to process this structure correctly.

# Assignments

1. Make an `Measurement` structure with two fields: the measured value `x` and its standard deviation `σ` (`\sigma<TAB>`). Make sure that both `x` and `σ` are of the same type and a subtype of `Real`.
2. Make a constructor for `Measurement`, such that an error is returned when a negative standard error is given.
3. Make two functions `val` and `err`, which respecitivly return the value and the standard error of a measurement.
4. Make a binary operator `±` (`\pm<TAB>`), such that `x ± σ` returns a new instance of the type `Measurement`. Try it on a value 4.0 with a standard error of 1.2.
5. Overload all functions of scalar multiplication, adding and substracting measurements and adding a constant such that they correctly process measurements. Try some examples.
6. Overload the power function `^` such that one can raise `Measurement`s to a power. Note the special case `(x ± σ)^0` = `one(x) ± zero(σ)`.
7. Run example to generate error functions for a whole range of functions using metaprogramming.
8. Solve the small exercise using data.

**Assignment 1 and 2**

In [1]:
struct Measurement{T<:AbstractFloat}
    x::T
    σ::T
    function Measurement(x::T, σ::T) where {T <: AbstractFloat}
        @assert σ ≥ 0
        new{T}(x, σ)
    end
end

**Assignment 3**

In [2]:
val(m::Measurement) = m.x
err(m::Measurement) = m.σ

err (generic function with 1 method)

**Assignment 4**

Don't forget to add type annotations!

In [3]:
±(x::AbstractFloat, σ::AbstractFloat) = Measurement(x, σ)

± (generic function with 1 method)

In [4]:
Base.show(io::IO, m::Measurement) = print("$(m.x) ± $(m.σ)")

In [5]:
3.0 ± 4.0

3.0 ± 4.0

**Assignment 5**

In [6]:
# scalar multiplication
Base.:*(a::Real, m::Measurement) = a * m.x ± abs(a) * m.σ
Base.:/(m::Measurement, a::Real) = a * m
# adding and substracting measurements
Base.:+(m1::Measurement, m2::Measurement) = (m1.x + m2.x) ± sqrt(m1.σ^2 + m2.σ^2)
Base.:-(m1::Measurement, m2::Measurement) = (m1.x + m2.x) ± sqrt(m1.σ^2 - m2.σ^2)
Base.:-(m::Measurement) = -m.x ± m.σ
# adding a constant
Base.:+(m::Measurement, a::Real) = (m.x + a) ± m.σ
Base.:+(a::Real, m::Measurement) = (m.x + a) ± m.σ
# multiplying two measurments
Base.:*(m1::Measurement, m2::Measurement) = (m1.x * m2.x) ± (m1.x * m2.x) * sqrt((m1.σ/m1.x)^2 + (m2.σ/m2.x)^2)

In [7]:
m1 = 1.2 ± 0.6
m2 = 2.1 ± 1.5

2.1 ± 1.5

In [8]:
2m1

2.4 ± 1.2

In [9]:
m1 + m2

3.3 ± 1.6155494421403511

In [10]:
m1 * m2

2.52 ± 2.1971800108320667

**Assignment 6**

In [11]:
Base.:^(m::Measurement{T}, p::Integer) where {T} = p == 0 ? one(T) ± zero(T) : m.x^p ± p*abs(m.x^(p-1)) * m.σ

We might just as well define division.

In [12]:
Base.:/(m1::Measurement, m2::Measurement) = m1 * (m2)^-1

In [13]:
m1^2

1.44 ± 1.44

In [14]:
m2^0

1.0 ± 0.0

**Assignment 7**

In [15]:
using ForwardDiff

for f in [:sin, :cos, :tan, :exp, :log, :log2, :log10, :sqrt, :inv]
    eval(quote
        # this is a line of code generated using string interpolation
        Base.$f(m::Measurement) = $f(m.x) ± abs(ForwardDiff.derivative($f, m.x) * m.σ)
    end)
end

**Assignment 8**

Let's apply this to in a somewhat realistic setting. Many methods in analytical chemistry are based on the law of [Beer-Lambert](https://en.wikipedia.org/wiki/Beer%E2%80%93Lambert_law). This law relates the absorption of a ray of light passing through a cuvet with the concentration of a solution. For a given reference intensity $I_0$ at concentration of 0 and a lower intensity $I$ when it passes through a solution of concentration $c$, we have

$$
\log \left(\frac{I_0}{I}\right) = \varepsilon c l\,,
$$

with $\varepsilon$ the molar extinction coefficient and $$l$$ the thickness of the cuvet.

Suppose we want to determine the extinction coefficient for some substance, using a cuvet of thickness $0.02\pm 0.001$ and a reference solution of a concentration of $0.73\pm 0.02$ M.

We perform some intensity measurements, with associated measurement errors.

| $I0$ | $\sigma_{I0}$ | $I1$ | $\sigma_{I1}$ |
|------|---------------|------|---------------|
| 0.8  | 0.14          | 0.2  | 0.12          |
| 1.1  | 0.11          | 0.3  | 0.07          |
| 1.2  | 0.08          | 0.4  | 0.101         |


**Estimate the molecular extinction coefficient.**

In [16]:
I0 = [0.8, 1.1, 1.2] .± [0.14, 0.11, 0.08]

3-element Array{Measurement{Float64},1}:
 
 
 

0.8 ± 0.141.1 ± 0.111.2 ± 0.080.8 ± 0.140.8 ± 0.141.1 ± 0.111.1 ± 0.111.2 ± 0.081.2 ± 0.08

In [17]:
I1 = [0.2, 0.3, 0.4] .± [0.12, 0.07, 0.101]

3-element Array{Measurement{Float64},1}:
 
 
 

0.2 ± 0.120.3 ± 0.070.4 ± 0.1010.2 ± 0.120.2 ± 0.120.3 ± 0.070.3 ± 0.070.4 ± 0.1010.4 ± 0.101

In [18]:
l = 0.02 ± 0.001
c = 0.73 ± 0.02

0.73 ± 0.02

In [19]:
mean(x) = sum(x) / length(x)

ϵ = log(mean(I0) / mean(I1)) / (c*l)

84.70976898280321 ± 14.587221174862679